In [1]:
import ipyvuetify as v
from sepal_ui import sepalwidgets as sw
from scripts.tile import *

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

In [3]:
class step4(sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Restoration goals"
        id_ = "nested_widget"
        
        goals = [
            'Enhancement of existing areas',
            'Increase the forest cover',
            'Reflect relevant national regulations',
            'Achievement of international commitments',
            'Improve connectivity - landscape biodiversity'
        ]
        
        select = v.Select(
            label = "restoration goals",
            items = goals,
            V_model = None
        )
        
        super().__init__(
            id_, 
            title, 
            inputs = [select],
            **kwargs
        )
        self.children[0].elevation = 0
        

In [4]:
class satisfaction_slider(v.Slider):
    
    _labels = [
        'no importance',
        'low importance',
        'neutral',
        'important',
        'very important',
    ]
        
    _colors = [
        'red',
        'orange',
        'yellow accent-3',
        'light-green',
        'green'
    ]
    
    def __init__(self, label, **kwargs):
        
        super().__init__(
            label = label,
            max = 4,
            min = 0,
            #step = 1,
            #ticks = "always",
            #tick_size = 4,
            track_color = 'grey',
            thumb_label = self._labels[4],
            color = self._colors[4],
            v_model = 4
        )
        
        self.observe(self.on_change, 'v_model')
        
    def on_change(self, change):
        self.color = self._colors[change['new']]
        self.thumb_label = self._labels[change['new']]

In [5]:
class step5 (sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Restoration priorities"
        id_ = "nested_widget"
        
        super().__init__(
            id_, 
            title, 
            inputs = [
                satisfaction_slider('Production Forest'),
                satisfaction_slider(' Protected Area'),
                satisfaction_slider('Community Foresty'),
                satisfaction_slider('Community Protected Area'),
                satisfaction_slider('Biodiversity conservation corridor'),
                satisfaction_slider('Other private land or public land')
            ],
            **kwargs
        )
        self.children[0].elevation = 0

In [6]:
step2_content = v.StepperContent(
    key=1,
    step=1, 
    children=[step2()]
)  

step3_content = v.StepperContent(
    key=2,
    step=2, 
    children=[step3()]
)

step4_content = v.StepperContent(
    key=3,
    step=3, 
    children=[step4()]
)

step5_content = v.StepperContent(
    key=4,
    step=4, 
    children=[step5()]
)

stepper_children = [
    v.StepperStep(
        key=1,
        complete=False,
        step=1,
        editable=True,
        children=["Constraints"]
    ),
    v.Divider(),
    v.StepperStep(
        key=2,
        complete=False,
        step=2,
        editable=True,
        children=["Restoration potential and land use"]
    ),
    v.Divider(),
    v.StepperStep(
        key=3,
        complete=False,
        step=3,
        editable=True,
        children=["Restoration goals"]
    ),
    v.Divider(),
    v.StepperStep(
        key=4,
        complete=False,
        step=4,
        editable=True,
        children=["Restoration priorities"]
    ),
]

stepper_header = v.StepperHeader(
    children=stepper_children
)

aoi_content = v.Stepper(
        class_="mt-2",
         children=[
             stepper_header,
             v.StepperItems(
                 children=[
                     step2_content,
                     step3_content,
                     step4_content,
                     step5_content
                 ]
             )
         ]
)

NameError: name 'step2' is not defined

In [ ]:
title = "Questionnaire Identifying Restoration Objective & Goals"
id_ = "questionnaire_widget"
process_tile = sw.Tile(id_, title, inputs=[aoi_content])
process_tile